# Contents
* [Intro](#Intro)
* [Imports and config](#Imports-and-config)
* [Load data](#Load-data)
* [Train test split](#Train-test-split)
* [Convolutional Neural Network](#Convolutional-Neural-Network)
  * [Ternary](#Ternary)
    * [Fit ternary](#Fit-ternary)
    * [Results ternary](#Results-ternary)
  * [Binary](#Binary)
      * [Fit binary](#Fit-binary)
      * [Results binary](#Results-binary)
* [Discussion](#Discussion)

## Intro

This notebook sets up a Convolutional Neural Network (CNN) to classify audio of short duration by extracted Mel Frequencey Cepstral Coefficients (MFCCs). Both ternary and binary classification are considered. In all cases except for the binary positive/non-positive case, the trained classifier was able to outperform the dummy classifier.

## Imports and config

In [1]:
# set seed
from numpy.random import seed

seed(SEED := 2021)

In [2]:
# Extensions
%load_ext lab_black
%load_ext nb_black
%load_ext autotime

In [3]:
# Core
import numpy as np
import pandas as pd
from collections import namedtuple
import swifter

# keras
from keras.models import Sequential
from keras.layers import (
    Conv2D,
    GlobalMaxPooling2D,
    Dense,
)
import tensorflow as tf

# display outputs w/o print calls
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

# suppress warnings
import warnings

warnings.filterwarnings("ignore")

time: 6.18 s


In [4]:
# Location of parquet
PARQUET_DF_FOLDER = "../5.0-mic-extract_spectrograms_and_MFCCs_short"

# Location where this notebook will output
DATA_OUT_FOLDER = "."

# The preprocessed data from the Unified Multilingual Dataset of Emotional Human utterances
WAV_DIRECTORY = (
    "../../../unified_multilingual_dataset_of_emotional_human_utterances/data/preprocessed"
)

time: 6 ms


## Load data

In [5]:
short_df = pd.read_parquet(f"{PARQUET_DF_FOLDER}/short_plus.parquet")
short_df.head(1)

,file,duration,source,speaker_id,speaker_gender,emo,valence,lang1,lang2,neg,neu,pos,length,padded,mfcc,melspec_db
0,01788+BAUM1+BAUM1.s028+f+hap+1+tur+tr-tr.wav,0.387,BAUM1,BAUM1.s028,f,hap,1,tur,tr-tr,0,0,1,short,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[-680.11646, -680.11646, -673.7514, -377.4224...","[[-80.0, -80.0, -80.0, -80.0, -80.0, -80.0, -7..."


time: 375 ms


## Train test split

The custom split ensures no data leakage due to speaker characteristics.

In [6]:
short_speakers = (
    pd.DataFrame(np.unique(short_df.speaker_id))
    .sample(frac=0.30, random_state=SEED)[0]
    .values
)

time: 9 ms


In [7]:
criterion = short_df.speaker_id.isin(short_speakers)
X_test = (_ := short_df.loc[criterion]).mfcc
y_test = _.valence
X_train = (_ := short_df.loc[~criterion]).mfcc
y_train = _.valence
len(short_df) == len(y_test) + len(y_train)
print(f"{len(y_test)} in test, {len(y_train)} in train")

True

190 in test, 290 in train
time: 18 ms


Some additional preprocessing is needed to format the data for keras.

In [8]:
y_test = tf.keras.utils.to_categorical(y_test, num_classes=3, dtype="float32")
y_train = tf.keras.utils.to_categorical(y_train, num_classes=3, dtype="float32")

stack = np.stack
reshaper: np.ndarray = lambda x: stack(x.swifter.apply(stack)).reshape(
    len(x), 16, 20, 1
)
X_train, X_test = reshaper(X_train), reshaper(X_test)

Pandas Apply: 100%|██████████| 190/190 [00:00<00:00, 18997.75it/s]

time: 67 ms


## Convolutional Neural Network

### Ternary

We will start with a simple CNN. After the input layer, there is one convolutional layer, a global max pooling layer, and a softmax output layer for three classes.

In [9]:
model_cnn = Sequential(
    [
        Conv2D(filters=64, kernel_size=2, activation="relu", input_shape=(16, 20, 1)),
        GlobalMaxPooling2D(),
        Dense(3, activation="softmax"),
    ]
)

model_cnn.compile(
    loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
)
model_cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 15, 19, 64)        320       
_________________________________________________________________
global_max_pooling2d (Global (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 3)                 195       
Total params: 515
Trainable params: 515
Non-trainable params: 0
_________________________________________________________________
time: 621 ms


#### Fit ternary

Let's see how well the ternary classifier works.

In [10]:
model_cnn.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=15,
    batch_size=1,
)

Epoch 1/15
290/290 [==============================] - 1s 2ms/step - loss: 9.9274 - accuracy: 0.4000 - val_loss: 4.3067 - val_accuracy: 0.3105
Epoch 2/15
290/290 [==============================] - 1s 2ms/step - loss: 4.8545 - accuracy: 0.4241 - val_loss: 3.6765 - val_accuracy: 0.3632
Epoch 3/15
290/290 [==============================] - 0s 2ms/step - loss: 4.4953 - accuracy: 0.4862 - val_loss: 2.9881 - val_accuracy: 0.3684
Epoch 4/15
290/290 [==============================] - 1s 2ms/step - loss: 3.1725 - accuracy: 0.4828 - val_loss: 6.5195 - val_accuracy: 0.1947
Epoch 5/15
290/290 [==============================] - 1s 2ms/step - loss: 3.3266 - accuracy: 0.5138 - val_loss: 3.6179 - val_accuracy: 0.3895
Epoch 6/15
290/290 [==============================] - 1s 2ms/step - loss: 2.7131 - accuracy: 0.5103 - val_loss: 3.5614 - val_accuracy: 0.4421
Epoch 7/15
290/290 [==============================] - 1s 2ms/step - loss: 3.0475 - accuracy: 0.4517 - val_loss: 2.8889 - val_accuracy: 0.3316
Epoch 

time: 8.43 s


#### Results ternary

How well would a dummy classifier do in ternary classification?

In [11]:
print("dummy classifer:")
len_full_test = len(X_test)
for valence in {"-1", "0", "1"}:
    test_valence_set = short_df.loc[
        (short_df.valence == valence) & short_df.speaker_id.isin(short_speakers)
    ]
    print(
        f"{(_ := len(test_valence_set.loc[test_valence_set.valence == valence]))} samples of valence {valence} in test split ({(__ := _ / len_full_test):.3f} / {1 - __:.3f})"
    )

dummy classifer:
39 samples of valence 1 in test split (0.205 / 0.795)
66 samples of valence -1 in test split (0.347 / 0.653)
85 samples of valence 0 in test split (0.447 / 0.553)
time: 23 ms


The best validation accuracy from the fifteen epochs above was about 55.3%. It outperformed the dummy classifier (best score of 44.7%) by about 10.6%.

### Binary

Now we will set up to run the same architecture with slight modifications for the binary cases. Namely, the output layer reflects the number of classes and uses a sigmoid activation function rather than softmax; also, the loss function was changed from categorical cross entropy to binary cross entropy.

In [12]:
OvrSet = namedtuple("OvrSet", "name, test, train, dummy")

time: 4 ms


In [13]:
binary_valence = [
    OvrSet(
        name=valence,
        test=(_ := short_df.loc[criterion][valence]),
        train=short_df.loc[~criterion][valence],
        dummy=_.swifter.apply(lambda _: _ == 1).sum() / len(_),
    )
    for valence in ("neg", "neu", "pos")
]

time: 21 ms


#### Fit binary

The following cell loops through the binary classification sets.

In [14]:
for ovr_set in binary_valence:
    dummy = ovr_set.dummy
    print("valence:", ovr_set.name, "dummy score:", dummy if dummy > 0.5 else 1 - dummy)
    model_cnn = Sequential(
        [
            Conv2D(
                filters=64, kernel_size=2, activation="relu", input_shape=(16, 20, 1)
            ),
            GlobalMaxPooling2D(),
            Dense(1, activation="sigmoid"),
        ]
    )

    model_cnn.compile(
        loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    model_cnn.summary()
    model_cnn.fit(
        X_train,
        ovr_set.train,
        validation_data=(X_test, ovr_set.test),
        epochs=15,
        batch_size=1,
    )

valence: neg dummy score: 0.6526315789473685
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 15, 19, 64)        320       
_________________________________________________________________
global_max_pooling2d_1 (Glob (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 385
Trainable params: 385
Non-trainable params: 0
_________________________________________________________________
Epoch 1/15
290/290 [==============================] - 1s 2ms/step - loss: 4.5500 - accuracy: 0.5828 - val_loss: 2.5321 - val_accuracy: 0.6895
Epoch 2/15
290/290 [==============================] - 1s 2ms/step - loss: 3.9348 - accuracy: 0.5759 - val_loss: 2.4665 - val_accuracy: 0.6842
Epoch 3/15
290/290 [==============================] - 0s 

valence: neu dummy score: 0.5526315789473684
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 15, 19, 64)        320       
_________________________________________________________________
global_max_pooling2d_2 (Glob (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 385
Trainable params: 385
Non-trainable params: 0
_________________________________________________________________
Epoch 1/15
290/290 [==============================] - 1s 2ms/step - loss: 4.0531 - accuracy: 0.5724 - val_loss: 2.5015 - val_accuracy: 0.5789
Epoch 2/15
290/290 [==============================] - 1s 2ms/step - loss: 1.9142 - accuracy: 0.6552 - val_loss: 1.4641 - val_accuracy: 0.6158
Epoch 3/15
290/290 [==============================] - 0s 

valence: pos dummy score: 0.7947368421052632
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 15, 19, 64)        320       
_________________________________________________________________
global_max_pooling2d_3 (Glob (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 385
Trainable params: 385
Non-trainable params: 0
_________________________________________________________________
Epoch 1/15
290/290 [==============================] - 1s 2ms/step - loss: 5.9032 - accuracy: 0.5966 - val_loss: 1.5506 - val_accuracy: 0.7737
Epoch 2/15
290/290 [==============================] - 1s 2ms/step - loss: 2.4259 - accuracy: 0.6586 - val_loss: 0.9686 - val_accuracy: 0.6105
Epoch 3/15
290/290 [==============================] - 1s 

time: 26.4 s


#### Results binary

In the negative/non-negative case, the dummy score on the test set was 65.3%, which underperformed the CNN classifier's best validation score of 69.0% by about 3.7%.

In the neutral/non-neutral case, the dummy score on the test set was 55.3%, which underperformed the CNN classifier's best validation score of 66.3% by about 11.1%.

In the positive/non-positive case, the dummy score on the test set was 79.5%, which was approximately equal to the classifier's best validation score.

## Discussion

The best-performing model in the ternary case as assessed by the best validation score of 15 epochs noticeably outperformed the dummy classifier, but the score just barely surpassed 55%.

In the binary cases, only the positive/non-positive classifier failed to surpass the dummy classifier's performance of the dummy classifier. This is the only case where the accuracy surpassed 70%. Class imbalance may play a role.

It may be better for the three one-vs-rest classifiers to share their lower layers; ensembling these would likely yield better performance.

Overall, the results are unremarkable, but there are many possible improvements to be considered. Firstly, we only have a few hundred observations in our subsample. Secondly, the architecture could be reconfigured with more layers and nodes.

[^top](#Contents)